<a href="https://colab.research.google.com/github/yeshurunlevi/popularity-of-restaurants-in-Greece/blob/main/popularity_of_restaurants_in_Greece.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

columns:

link - ID?

name - name

original_location - list of all

claimed - restaurant business claimed on TripAdvisor

awards - awards, when get.

popularity_detailed - popularity detailed ranking

popularity_generic - popularity generic ranking (among all places to eat in the area)

top_tags - top tag names (key words?)

price_level - level of prices in current currency (data scraped from www.tripadvisor.ie - so currency is €)

price_range - range of prices in current currency (ranges displayed on TripAdvisor seem to be quite unreliable)

meals - types of meal

cuisines - types of cuisine

special_diets - types of special diets

features - restaurant features

vegetarian_friendly - is the restaurant vegetarian friendly?

vegan_options - does the restaurant offer vegan options?

gluten_free - does the restaurant gluten-free options?

In [49]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

In [50]:
df = pd.read_csv(r'/content/drive/MyDrive/Trip Advisor EU restaurants /tripadvisor_european_restaurants.csv')

<ipython-input-50-c856f44723e1>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'/content/drive/MyDrive/Trip Advisor EU restaurants /tripadvisor_european_restaurants.csv')


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1083397 entries, 0 to 1083396
Data columns (total 42 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   restaurant_link                    1083397 non-null  object 
 1   restaurant_name                    1083396 non-null  object 
 2   original_location                  1083397 non-null  object 
 3   country                            1083397 non-null  object 
 4   region                             1033074 non-null  object 
 5   province                           742765 non-null   object 
 6   city                               682712 non-null   object 
 7   address                            1083397 non-null  object 
 8   latitude                           1067607 non-null  float64
 9   longitude                          1067607 non-null  float64
 10  claimed                            1081555 non-null  object 
 11  awards                  

In [52]:
df2 = df.copy()
df2 = df2[(df['total_reviews_count'] >= 10) & (df['country'] == 'Greece')]

#df2 = df2.applymap(lambda x: x.replace(" ", "") if isinstance(x, str) else x)
df2 = df2.applymap(lambda x: x.lower() if isinstance(x, str) else x)


def succ(x):
  if x >= 4.5:
    return 1
  else:
    return 0
df2['success (target)'] = [succ(x) for x in df2['avg_rating']]

<ipython-input-52-f49f0068eb8d>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2 = df2.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [53]:
df2['success (target)'].value_counts()

,count
success (target),
1,13796
0,8194


In [54]:
# Extract only the ranking number, handling NaNs
df2["popularity_detailed"], df2["restaurants"] = df["popularity_detailed"].str.extract(r'#(\d)'), df["popularity_detailed"].str.extract(r'#(\d+)')
df2["popularity_generic"], df2["places"] = df["popularity_generic"].str.extract(r'#(\d)'), df["popularity_generic"].str.extract(r'#(\d+)')

df2["popularity_detailed"], df2["restaurants"], df2["popularity_generic"], df2["places"] = df2["popularity_detailed"].astype(float), df2["restaurants"].astype(float), df2["popularity_generic"].astype(float), df2["places"].astype(float)

In [55]:

df2['price_level'] = df['price_level'].map({'€': 1, '€€-€€€': 2.5, '€€€€': 4}).astype(float)

df2['claimed'] = df['claimed'].map({'Claimed': 1, 'Unclaimed': 0}).astype(float)

df2['default_language'] = df['default_language'].map({'English': 0, 'All languages': 1}).astype(float)

In [56]:
# Days of the week
days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

# Function to process opening hours
def process_hours(open_hours):
    # If the entry is null, return a dictionary with all -1
    if pd.isna(open_hours):
        return {f"{day}_op": -1 for day in days} | {f"{day}_clo": -1 for day in days}

    # Convert string to dictionary
    try:
        hours_dict = ast.literal_eval(open_hours)  # Convert string to dictionary
    except:
        return {f"{day}_op": -1 for day in days} | {f"{day}_clo": -1 for day in days}

    # Create a dictionary to store results
    result = {}

    for day in days:
        if day in hours_dict and hours_dict[day]:  # If the day has hours
            open_time, close_time = hours_dict[day][0].split('-')  # Extract first range
            open_hour = int(open_time.split(':')[0])  # Get hour (HH)
            close_hour = int(close_time.split(':')[0])  # Get hour (HH)
        else:
            open_hour, close_hour = -1, -1  # If closed

        result[f"{day}_op"] = open_hour
        result[f"{day}_clo"] = close_hour

    return result

# Apply the function to the column and expand results into new columns
df_expanded = df2['original_open_hours'].apply(process_hours).apply(pd.Series)

# Merge back into the original DataFrame (optional)
df2 = pd.concat([df2, df_expanded], axis=1).drop(columns=['original_open_hours'])


In [57]:
def extract_prices(price_range):
    if pd.isna(price_range):
        return np.nan, np.nan  # Keep NaN for missing values
    else:
      prices = price_range.replace("€", "").replace(",", "").split("-")  # Remove € and split
    return [int(prices[0]), int(prices[1])]  # Convert to integers

# Apply function and create new columns
df2["min_price"] =[extract_prices(cell)[0] for cell in df2["price_range"]]
df2["max_price"] =[extract_prices(cell)[1] for cell in df2["price_range"]]


In [58]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21990 entries, 952974 to 986735
Data columns (total 60 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   restaurant_link                    21990 non-null  object 
 1   restaurant_name                    21990 non-null  object 
 2   original_location                  21990 non-null  object 
 3   country                            21990 non-null  object 
 4   region                             21990 non-null  object 
 5   province                           18520 non-null  object 
 6   city                               21987 non-null  object 
 7   address                            21990 non-null  object 
 8   latitude                           21340 non-null  float64
 9   longitude                          21340 non-null  float64
 10  claimed                            21983 non-null  float64
 11  awards                             10731 non-null  ob

In [59]:
count_awards(df2['awards'])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [66]:

df2['awards'].fillna(' ', inplace=True)  # Replace NaN

# Apply function and expand into new columns
df2['travellers_choice'] = [len(re.findall(r"travellers' choice", cell, re.IGNORECASE)) for cell in df2['awards']]
df2['excellence'] = [len(re.findall(r"excellence", cell, re.IGNORECASE)) for cell in df2['awards']]
df2['michelin'] = [len(re.findall(r"michelin", cell, re.IGNORECASE)) for cell in df2['awards']]
df2['another'] = [len(re.findall(r"worth", cell, re.IGNORECASE)) + len(re.findall(r"good", cell, re.IGNORECASE)) for cell in df2['awards']]

In [70]:
# Function to clean and split addresses
def clean_and_split_address(address):
    address = address.lower().replace("greece", "").strip()  # Remove "Greece"

    # Ensure we have at least a street and city/zip
    parts = address.split(",", 1) if "," in address else [address, ""]

    street_info = parts[0].strip()  # Street + number
    city_zip = parts[1].strip() if len(parts) > 1 else ""  # City + ZIP if available

    # Extract ZIP code and city
    zip_code = []
    city = []
    for char in city_zip:
        if char.isdigit():
            zip_code.append(char)
        else:
            city.append(char)

    city = "".join(city).strip()
    zip_code = "".join(zip_code)

    # Extract Apartment Number and Street Name
    apartment_number = []
    street_name = []
    for char in street_info:
        if char.isdigit():
            apartment_number.append(char)
        else:
            street_name.append(char)

    street_name = "".join(street_name).strip()
    apartment_number = "".join(apartment_number)

    return pd.Series([apartment_number, street_name, city, zip_code])

# Apply function to DataFrame
df2[['Apartment Number', 'Street Name', 'City_2', 'Zip Code']] = df2['address'].apply(clean_and_split_address)


# Display results
df2[['Apartment Number', 'Street Name', 'City_2', 'Zip Code']]


,Apartment Number,Street Name,City_2,Zip Code
952974,13,sekeri,tolon,21056
952975,14,atkis st,tolon,21056
952976,59,sekeri,tolon,21056
952977,8,aktis,tolon,21056
952978,28,sekeri,tolon,21056
...,...,...,...,...
986731,,fodele,crete,71500
986732,,kalamaki,"chania town, crete",73100
986733,5,th km chania-kissamos old road,"kalamaki, chania town, crete",73500
986734,,peo kissamou chanion kalamaki beach,"kalamaki, chania town, crete",73100


In [71]:
import pandas as pd
from collections import Counter


# Function to split awards and count them
def count_awards(awards_column):
    # Split each entry in the column by commas and flatten the result
    all_awards = [award.strip() for entry in awards_column if type(entry) == str for award in entry.split(',')]

    # Use Counter to count occurrences of each award
    award_count = Counter(all_awards)

    return award_count


In [72]:
def extract_city(address):
    if isinstance(address, str):  # Check if the value is a string
        parts = address.split(",", 1)  # Split at the first comma
        return parts[0].strip()  # Return the part before the comma
    return ""  # Return an empty string if the address is not a string

# Fill NaN values in 'city' column with values from 'new city' column
df2['city'] = df2['city'].fillna(df2['City_2'].apply(extract_city))

In [74]:
df2['to_new_data'] = df2['top_tags'] + ',' +df2['meals']+ ',' + df2['cuisines'] + ',' + df2['special_diets']+ ',' + df2['features']+ ','+ df2['keywords']

import pandas as pd
from collections import Counter


# Function to split awards and count them
def count_awards(awards_column):
    # Split each entry in the column by commas and flatten the result
    all_awards = [award.strip() for entry in awards_column if type(entry) == str for award in entry.split(',')]

    # Use Counter to count occurrences of each award
    award_count = Counter(all_awards)

    return award_count

# Apply the function
top_counts = count_awards(df2['to_new_data'])

valid_terms = {k: v for k, v in top_counts.items() if v >= 100}# threshold

# Ensure 'to_new_data' is treated as a string before applying .lower()
for term in valid_terms.keys():
    df2[term] = df2['to_new_data'].astype(str).apply(lambda x: 1 if term in x.lower() else 0)

In [75]:
#to drop
droping = ['restaurant_link', 'original_location','country', 'restaurant_link', 'awards', 'vegetarian_friendly', 'vegan_options', 'gluten_free', 'top_tags', 'meals'
, 'cuisines', 'special_diets', 'features', 'keywords', 'price_range', 'address', 'City_2']
df2 = df2.drop(columns=droping, axis=1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!apt-get install git